In [1]:
import tensorflow as tf
from tensorflow.compat.v1.keras import backend as K

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Input
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
config=tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.log_device_placement=True
sess=tf.compat.v1.Session(config=config)
K.set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5



In [4]:
import pandas as pd
from collections import Counter
import numpy as np
import re
import os
from numpy import zeros
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [5]:
data3=pd.read_csv('occ_categories_nouns_only.csv')
data3.head()

,Unnamed: 0,occ_group,skill_category,skill_title,job_description,city,state,no_stopwords_stemmed,data_len
0,0,basic,Mathematics.xls,Mathematicians,Duties \n Summary \n About the Position: This...,None,None,duti posit posit duti locat applic level targe...,796
1,1,basic,Mathematics.xls,Mathematicians,Senior Business Analyst / Lead Business Analys...,New York,NY,busi analyst busi analyst organ group chegg gr...,309
2,2,basic,Mathematics.xls,Mathematicians,"Manager, Business Analytics \nAnalytics | New ...",New York,NY,manag organ group chegg group data decis chegg...,238
3,3,basic,Mathematics.xls,Mathematicians,About Mastery: \nWe believe educational inequi...,Camden,NJ,masteri inequ problem countri right issu day m...,211
4,4,basic,Mathematics.xls,Mathematicians,About Mastery: \nWe believe educational inequi...,Philadelphia,PA,masteri inequ problem countri right issu day m...,211


In [6]:
len(data3)

22533

In [7]:
%%time
collection=[]
for i in data3.no_stopwords_stemmed:
    collection.append(re.findall(r'http://[a-z0-9_.\?=&-@+]{8,150}|https://[a-z0-9_.\?=&-@+]{8,150}', str(i)))

CPU times: user 38 ms, sys: 457 µs, total: 38.5 ms
Wall time: 37.6 ms


In [8]:
%%time
X_emails__http_rm=[]
for i,j in zip(data3.no_stopwords_stemmed,collection):
    X_emails__http_rm.append(' '.join([p for p in i.split() if p not in j]))   

CPU times: user 497 ms, sys: 6.72 ms, total: 504 ms
Wall time: 504 ms


In [9]:
data3['X_emails__http_rm']=pd.Series(X_emails__http_rm)

In [10]:
len(X_emails__http_rm)

22533

In [11]:
groups=data3[['occ_group', 'X_emails__http_rm']]
groups.head()

,occ_group,X_emails__http_rm
0,basic,duti posit posit duti locat applic level targe...
1,basic,busi analyst busi analyst organ group chegg gr...
2,basic,manag organ group chegg group data decis chegg...
3,basic,masteri inequ problem countri right issu day m...
4,basic,masteri inequ problem countri right issu day m...


In [12]:
len(groups)

22533

In [13]:
d_group=groups.groupby('occ_group')['X_emails__http_rm'].apply(' '.join).reset_index()
d_group.head()

,occ_group,X_emails__http_rm
0,basic,duti posit posit duti locat applic level targe...
1,complex problem solving,career state branch agenc state workforc emplo...
2,resource management,locat job posit summari technolog offic experi...
3,social,qualiti summer program school readi practic li...
4,systems,descript test administr leader comput technolo...


In [14]:
def tf_idf_by_category(text, categories):
    N=len(text)

    counts2=[Counter(i.split()) for i in text]

    unique_word_dict=dict(pd.Series(' '.join(text).split()).value_counts())

    D=list(unique_word_dict.keys())

    token_text=[]
    for i in text:
        token_text.append([p for p in i.split()])

    tf_dict=[]
    for i in token_text:
        dic=dict(pd.Series(i).value_counts())

        tf_dict.append([[p,dic[p]/len(i)] for p in i])

    tf_={}
    for i,k in zip(categories,tf_dict):
        tf_[i]=[p[1] for p in k]


    tf_list=list(tf_.values())


    enclud=[]
    for d in D:
        for i,k in enumerate(counts2):
            if d in k:
                enclud.append([d,i])

    a=[]
    for i in enclud:
        a.append(i[0])

    enclude_dict=dict(pd.Series(a).value_counts())   

    idf={}
    for i,k in enclude_dict.items():
        idf[i]=np.log(N/k)

    embedded_idf=[]
    for i in token_text:
        try:
            embedded_idf.append([idf[j] for j in i])

        except KeyError:
            continue 
            
    group_tf_idf=[dict(zip(t,(np.multiply(p,l)))) for t,p,l in zip(token_text,tf_list,embedded_idf)]
    
    skill_dict=dict(zip(categories,group_tf_idf))
    
    return skill_dict

In [15]:
%%time
category_td_idf_dic=tf_idf_by_category(d_group.X_emails__http_rm, d_group.occ_group)


CPU times: user 11.1 s, sys: 315 ms, total: 11.4 s
Wall time: 11.4 s


In [16]:
embedded_skill=[]
for i in d_group.occ_group:
    for p in data3[data3['occ_group']==i].X_emails__http_rm:
        embedded_skill.append([category_td_idf_dic[i][j] for j in p.split()])
 

In [17]:
trial=pd.concat([data3.occ_group,data3.skill_category,data3.skill_title, 
                 data3.no_stopwords_stemmed,pd.Series([np.mean(i) for i in embedded_skill])], axis=1)

In [18]:
#trial.to_csv('trail4.csv')

In [19]:
embed_index=dict()
glove_dir=open(os.path.join('glove.6B/glove.6B.100d.txt'), encoding='utf8')

In [20]:
%%time
for line in glove_dir:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embed_index [word] = vector_dimensions

glove_dir.close()

CPU times: user 7.06 s, sys: 155 ms, total: 7.22 s
Wall time: 7.96 s


In [21]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(data3.X_emails__http_rm)

In [22]:
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

18419

In [23]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embed_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [24]:
len_sent=[len(i) for i in embedded_skill]

In [25]:
length_long_sentence=max(len_sent)
length_long_sentence

1247

In [26]:
padded_sentences = pad_sequences(embedded_skill, length_long_sentence, padding='post')

In [27]:
padded_sentences.shape

(22271, 1247)

In [28]:
validation_split=.2
indices=np.arange(np.array(padded_sentences,dtype=object).shape[0])
np.random.shuffle(indices)

In [29]:
indices.shape

(22271,)

In [30]:
indices

array([ 7457,  5974,  2642, ..., 13760, 15781,  4106])

In [31]:
data_rand=padded_sentences[indices]

In [32]:
data_rand.shape

(22271, 1247)

In [33]:
labe=[str(i).replace('.xls','') for i in data3.occ_group]
pd.Series(labe).unique()

array(['basic', 'social', 'technical', 'resource management', 'systems',
       'complex problem solving', 'nan'], dtype=object)

In [34]:
labels_encoded=le.fit(labe)
labels_=le.transform(labe)

In [35]:
labels_

array([0, 0, 0, ..., 2, 2, 2])

In [36]:
labels_rand=np.array(labels_)[indices]

In [37]:
labels_rand

array([6, 5, 4, ..., 0, 5, 3])

In [38]:
val_sample=int(validation_split * data3.shape[0])

In [39]:
val_sample

4506

In [40]:
X_train=data_rand[:-val_sample]
y_train=labels_rand[:-val_sample]
X_test=data_rand[-val_sample:]
y_test=labels_rand[-val_sample:]

In [41]:
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (17765, 1247)
y_train shape: (17765,)
X_test shape: (4506, 1247)
y_test shape: (4506,)


In [42]:
length_long_sentence

1247

In [62]:
deep_inputs = Input(shape=(length_long_sentence,))
embedding = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)(deep_inputs) 
flatten = Flatten()(embedding)
#dropout = Dropout(.05)(flatten)
initial = Dense(15, activation='relu')(flatten)
output=Dense(1, activation='sigmoid')(initial)
model = Model(inputs=deep_inputs, outputs=output)

In [63]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)


In [64]:
#model.compile(optimizer='sgd', loss='categorical_hinge', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule), loss='categorical_hinge', metrics=['accuracy'])
print(model.summary())

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1247)]            0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1247, 100)         1841900   
_________________________________________________________________
flatten_4 (Flatten)          (None, 124700)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 15)                1870515   
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 16        
Total params: 3,712,431
Trainable params: 1,870,531
Non-trainable params: 1,841,900
_________________________________________________________________
None


In [65]:
model.fit(X_train, y_train, epochs=50, verbose=1,batch_size = 20)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

print('Accuracy: %f' % (accuracy*100))

Epoch 1/50
889/889 [==============================] - 3s 4ms/step - loss: 0.3450 - accuracy: 0.2185
Epoch 2/50
889/889 [==============================] - 3s 3ms/step - loss: 0.3296 - accuracy: 0.2203
Epoch 3/50
889/889 [==============================] - 3s 3ms/step - loss: 0.3118 - accuracy: 0.2169
Epoch 4/50
889/889 [==============================] - 3s 3ms/step - loss: 0.3010 - accuracy: 0.2163
Epoch 5/50
889/889 [==============================] - 3s 3ms/step - loss: 0.2975 - accuracy: 0.2175
Epoch 6/50
889/889 [==============================] - 3s 3ms/step - loss: 0.3015 - accuracy: 0.2228
Epoch 7/50
889/889 [==============================] - 3s 4ms/step - loss: 0.2951 - accuracy: 0.2155
Epoch 8/50
889/889 [==============================] - 3s 4ms/step - loss: 0.3012 - accuracy: 0.2194
Epoch 9/50
889/889 [==============================] - 3s 3ms/step - loss: 0.3049 - accuracy: 0.2240
Epoch 10/50
889/889 [==============================] - 3s 3ms/step - loss: 0.2986 - accuracy: 0.2200

In [319]:
'''dropout1 = Dropout(.02)(initial)
hidden = Dense(15, activation='softsign')(dropout1)
hidden2= Dense(15, activation='sigmoid')(hidden)
dropout2 = Dropout(.02)(hidden2)
dropout = Dropout(.05)(flatten)
initial = Dense(42, activation='relu')(dropout)
dropout1 = Dropout(.05)(initial)'''